    THIS IS THE PYTHON NOTEBOOK WHERE ANALYSIS WAS DONE ON THE FOLLOWING :

1) Structure of PDFs. To find what is skeleton of each pdf. Is there anything common between them?
2) To find what fonts, tags and font size has been used.


     
    Following contains my analysis on 'paper1.pdf'


RESULT : THIS ANALYSIS PLAYED A VERY IMPORTANT ROLE IN CORRECTLY IDENTIFYING HEADERS AND FOOTERS. HELPED US IN ISOLATING THEM.


In [2]:
from operator import itemgetter
import fitz
import json
import re                          #importing regular expression library 


'''function to find all font sizes used in the pdf and how many times (count)
This funtion will tell most associated font style with a particular font size'''
def fonts(doc):                             

    styles = {}
    font_counts = {}
    for page in doc:
        blocks = page.get_text("dict")["blocks"]
        for b in blocks:  # iterate through the text blocks
            if b['type'] == 0:  # block contains text
                for l in b["lines"]:  # iterate through the text lines
                    for s in l["spans"]:  # iterate through the text spans
                        
                        identifier = "{0}".format(s['size'])
                        styles[identifier] = {'size': s['size'], 'font': s['font']}

                        font_counts[identifier] = font_counts.get(identifier, 0) + 1  # count the fonts usage

    font_counts = sorted(font_counts.items(), key=itemgetter(1), reverse=True)

    if len(font_counts) < 1:
        raise ValueError("No font found!")

    return font_counts, styles

'''Funtion to identify relation between font sixe and tags used'''

def font_tags(font_counts, styles):

    p_style = styles[font_counts[0][0]]  # get style for most used font by count (paragraph)
    p_size = p_style['size']  # get the paragraph's size

    # sorting the font sizes high to low, so that we can append the right integer to each tag
    font_sizes = []
    for (font_size, count) in font_counts:
        font_sizes.append(float(font_size))
    font_sizes.sort(reverse=True)

    # aggregating the tags for each font size
    idx = 0
    size_tag = {}
    for size in font_sizes:
        idx += 1
        if size == p_size:
            idx = 0
            size_tag[size] = '<p>'
        if size > p_size:
            size_tag[size] = '<h{0}>'.format(idx)
        elif size < p_size:
            size_tag[size] = '<s{0}>'.format(idx)

    return size_tag


'''Function to scrape pdf and return text elements with tags'''
def headers_para(doc, size_tag):

    header_para = []  # list with headers and paragraphs
    foots = []
    Raw_data = []
    first = True  # boolean operator for first header
    previous_s = {} # previous span
    abstract = []

    page_attribute = {}

    for pagenum, page in enumerate(doc):
        foots = []
        Raw_data = []
        
        blocks = page.get_text("dict")["blocks"]
        for b in blocks:  # iterate through the text blocks
            if b['type'] == 0:  # this block contains text

                block_string = ""  # text found in block
                for l in b["lines"]:  # iterate through the text lines
                    for s in l["spans"]:  # iterate through the text spans
                        if s['text'].strip():  # removing whitespaces:
                            if first:
                                previous_s = s
                                first = False
                                block_string = size_tag[s['size']] + s['text']
                            else:
                                if s['size'] == previous_s['size']:

                                    if block_string and all((c == "|") for c in block_string):
                                        # block_string only contains pipes
                                        block_string = size_tag[s['size']] + s['text']
                                    if block_string == "":
                                        # new block has started, so append size tag
                                        block_string = size_tag[s['size']] + s['text']
                                    else:  # in the same block, so concatenate strings
                                        block_string += " " + s['text']

                                else:
                                    header_para.append(block_string)
                                    block_string = size_tag[s['size']] + s['text']

                                previous_s = s

                    # new block started, indicating with a pipe
                        
                    block_string += " "

                if re.match(r'^<s4>', block_string):
                        block_string= re.sub(r'^<s4>', '', block_string, flags=re.MULTILINE)
                        foots.append(block_string)

                elif re.match(r'^<p>', block_string):
                        
                        block_string= re.sub(r'^<p>', '', block_string, flags=re.MULTILINE)
                        Raw_data.append(block_string)

                header_para.append(block_string)

            
        mapkey = 'Page_'+str(pagenum)
        page_attribute[mapkey] = {'footer': foots}

    return header_para,foots,Raw_data,abstract,page_attribute


def main():

    document = 'pdfs/paper1.pdf'
    doc = fitz.open(document)
    font_counts, styles = fonts(doc)
    size_tag = font_tags(font_counts, styles)


    print(font_counts)
    print(styles)
    print(size_tag)



if __name__ == '__main__':
     main()

[('11.9552001953125', 3622), ('7.970099925994873', 940), ('10.678384780883789', 180), ('5.97760009765625', 136), ('10.909099578857422', 120), ('7.665040016174316', 115), ('9.962599754333496', 87), ('6.973800182342529', 56), ('7.473479747772217', 43), ('6.191279888153076', 33), ('14.346199989318848', 31), ('11.14452075958252', 25), ('9.705060958862305', 16), ('9.774545669555664', 13), ('17.21540069580078', 13), ('4.981299877166748', 7), ('12.0302095413208', 3), ('9.02265739440918', 3), ('5.605109691619873', 3), ('16.54153823852539', 2), ('20.662500381469727', 2), ('22.556642532348633', 1), ('7.429680347442627', 1)]
{'16.54153823852539': {'size': 16.54153823852539, 'font': 'NimbusSans-Regular'}, '22.556642532348633': {'size': 22.556642532348633, 'font': 'TimesNewRomanPSMT'}, '12.0302095413208': {'size': 12.0302095413208, 'font': 'NimbusSans-Regular'}, '9.774545669555664': {'size': 9.774545669555664, 'font': 'NimbusSans-Regular'}, '9.02265739440918': {'size': 9.02265739440918, 'font': 'Ni

EXPLAINATION : AS WE CAN SEE THAT FONT SIZE 11 HAS BEEN USED MOST NUMBER OF TIMES THAT IS 3622.

Now this means it is most probably font size of the main body of the pdf. 

On further analysis we find that <s4> tag that uses font = (9.962599754333496) is where our footer resides. On the basis of this analysis we were able to device our OWN method to extract footers from pdf without fail BASED ON FONT SIZE AND TAG
